In [2]:
using Distributions
using PyPlot
include("utils.jl")

srand(42);

In [3]:
n = 1
γ = 0.1
σ = 1
d = dnorm(0, σ^2)
w = MvNormal(γ^2*eye(2))

data = rand(100) * 10
model = m -> x -> m[1] * x + m[2];
real_parameters = [ 1, 1 ]
observations = model(real_parameters).(data) + σ*randn(100);

incremental_likelihood = (m, n) -> exp(sum(log.([ d(m(data[i]) - observations[i]) for i in 1:n ])))

(::#13) (generic function with 1 method)

In [7]:
ion(); pygui(true);

# initialization

N = 100
X = rand(N, 2)
W = ones(N)/N

scatter(X[:,1], X[:, 2], s=W*N*5)
sleep(0.1)

# sampling
for n = 1:100
    # resampling
    if 1 / vecdot(W,W) < N/2
        X = X[rand(Categorical(W), N), :]
        W = ones(N)/N
    end

    Wₙ = [ incremental_likelihood(model(X[i,:]), n) for i in 1:N]
    W = W.*Wₙ / vecdot(W, Wₙ)

    X += rand(w, N)'

    m = vecdot(W, X[:,1]), vecdot(W, X[:, 2])
    clf()
    xlim(0, 2)
    ylim(0, 2)
    title("θ ≈ $m")
    scatter(X[:,1], X[:, 2], s=W*N*3, alpha=0.8, c="black")
    scatter([m[1]], [m[2]], c="red")
    scatter([real_parameters[1]], [real_parameters[2]], c="green")
    sleep(0.1)
end


In [8]:
# initialization
for N in [100; 500; 1000; 5000; 10000]
    X = rand(N, 2)
    W = ones(N)/N

    # sampling
    for n = 1:100
        # resampling
        if 1 / vecdot(W,W) < N/2
            X = X[rand(Categorical(W), N), :]
            W = ones(N)/N
        end

        Wₙ = [ incremental_likelihood(model(X[i,:]), n) for i in 1:N]
        W = W.*Wₙ / vecdot(W, Wₙ)

        X += rand(w, N)'
     end
    
    m = vecdot(W, X[:,1]), vecdot(W, X[:, 2])
    println(N, " ≈ $m")
end

100 ≈ (1.0246138266294476,0.878150285025453)
500 ≈ (0.9864145143413657,0.9183016751355478)
1000 ≈ (0.9998992076375581,0.8942803026115865)
5000 ≈ (1.003313202578945,0.9034339793593547)
10000 ≈ (1.0043169259479496,0.8939716489679784)
